In [98]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

In [99]:
# Download NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [100]:
df=pd.read_csv("1429_1.csv")

/tmp/ipython-input-450841150.py:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("1429_1.csv")


In [101]:
print(df.head())

                     id                                               name  \
0  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
1  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
2  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
3  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   
4  AVqkIhwDv8e3D1O-lebb  All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...   

        asins   brand                                         categories  \
0  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
1  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
2  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
3  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   
4  B01AHB9CN2  Amazon  Electronics,iPad & Tablets,All Tablets,Fire Ta...   

                                                keys manufacturer  \
0  84

In [102]:
df.shape

(34660, 21)

In [103]:


df=df[(df['categories'] == 'Electronics,iPad & Tablets,All Tablets,Fire Tablets,Tablets,Computers & Tablets') & (df['reviews.rating'] == 5)]

print(df.shape)

(1878, 21)


In [104]:
# print(df.columns.tolist())



In [105]:
text = " ".join(df['reviews.text'].dropna().astype(str))
print(text)

This product so far has not disappointed. My children love to use it and I like the ability to monitor control what content they see with ease. great for beginner or experienced person. Bought as a gift and she loves it Inexpensive tablet for him to use and learn on, step up from the NABI. He was thrilled with it, learn how to Skype on it already... I bought this for my grand daughter when she comes over to visit. I set it up with her as the user, entered her age and name and now Amazon makes sure that she only accesses sites and content that are appropriate to her age. Simple to do and she loves the capabilities. I also bought and installed a 64gig SD card which gives this little tablet plenty of storage. For the price I think this tablet is best one out there. You can spend hundreds of dollars more for additional speed and capacity but when it comes to the basics this tablets does everything that most people will ever need at a fraction of the cost. This amazon fire 8 inch tablet is 

In [106]:
!pip install emoji --quiet

In [107]:

import re
import emoji

def preprocess_review(text):
    # 1. Remove first-person phrases
    first_person_patterns = [
        r"\bI\b", r"\bwe\b", r"\bmy\b", r"\bour\b", r"\bme\b",
        r"\bmine\b", r"\bus\b"
    ]
    for pattern in first_person_patterns:
        text = re.sub(pattern, "", text, flags=re.IGNORECASE)

    # 2. Remove URLs and HTML tags
    text = re.sub(r"http\S+|www\S+", "", text)  # URLs
    text = re.sub(r"<.*?>", "", text)           # HTML tags

    # 3. Remove emojis
    text = emoji.replace_emoji(text, replace="")  # remove all emojis

    # 4. Remove ad phrases or repetitive boilerplate
    ad_phrases = [
        "Black Friday", "best buy", "craigslist", "Amazon", "sale", "free shipping",
        "click here", "buy now"
    ]
    for phrase in ad_phrases:
        text = re.sub(re.escape(phrase), "", text, flags=re.IGNORECASE)

    # 5. Remove extra whitespace, newlines, special characters
    text = re.sub(r"\s+", " ", text)        # multiple spaces → single space
    text = re.sub(r"[^A-Za-z0-9.,!?;:'\"()\s]", "", text)  # keep only text/punctuation
    text = text.strip()

    return text

df['reviews.text'] = df['reviews.text'].apply(preprocess_review)

In [108]:
df = df[df["reviews.text"].str.split().str.len() > 10]
df = df[df["reviews.text"].str.split().str.len() < 1024]


In [109]:
df = df.drop_duplicates(subset="reviews.text")

In [110]:
# from sentence_transformers import SentenceTransformer, util
# import pandas as pd
# import numpy as np

# # Load your dataset
# # df = df['reviews.text']  # must have a 'review' column
# all_reviews_text = " ".join(df['reviews.text'].dropna().astype(str))
# # Load DistilBERT-based embedding model
# model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# def extractive_summary(text, top_n=5):
#     sentences = text.split(". ")
#     sentences = [s.strip() for s in sentences if s]
#     embeddings = model.encode(sentences)
#     doc_emb = np.mean(embeddings, axis=0)
#     similarities = [util.cos_sim(doc_emb, emb)[0][0].item() for emb in embeddings]
#     top_idx = np.argsort(similarities)[-top_n:]
#     return ". ".join([sentences[i] for i in sorted(top_idx)])

# # Apply summarization
# summarize_text= extractive_summary(all_reviews_text, top_n=5)
# print(summarize_text)

In [111]:
# from transformers import pipeline
# summarizer = pipeline("summarization", model="google/flan-t5-large")
# all_reviews_text = " ".join(df['reviews.text'].dropna().astype(str))
# summary = summarizer(
#     "summarize: " + all_reviews_text,
#     max_length=150,
#     min_length=40,
#     do_sample=False
# )
# print(summary[0]['summary_text'])


In [137]:
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import numpy as np
import nltk

nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# -------------------------------
# Step 1: Extractive Summarization
# -------------------------------
prompt_text = (
    "Add product name and Summarize the following review in 3 sentences. "
    "Write in an objective tone and do not use first-person words like 'I' or 'we'. "
    "Focus on the main points and overall sentiment."
)

text = " ".join(df['reviews.text'].dropna().astype(str))
print(text)

# Split into sentences
sentences = sent_tokenize(text)



# Get sentence embeddings
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')
sentence_embeddings = embedder.encode(sentences, convert_to_tensor=True)

# Compute similarity with the "document centroid"
# document_embedding = np.mean(sentence_embeddings.cpu().numpy(), axis=0)
document_embedding = sentence_embeddings.mean(dim=0, keepdim=True)

cosine_scores = util.cos_sim(sentence_embeddings, document_embedding)

# Pick top-N most representative sentences
top_n = 10
top_sentence_indices = np.argsort(-cosine_scores.squeeze().cpu().numpy())[:top_n]
selected_sentences = [sentences[i] for i in sorted(top_sentence_indices)]

extractive_summary = " ".join(selected_sentences)
print("Extractive Summary:\n", extractive_summary)

# -------------------------------
# Step 2: Abstractive Summarization
# -------------------------------
base_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
model = PeftModel.from_pretrained(base_model, "lora-bart-cnn-adapter")
summarizer = pipeline("summarization", model=model)

final_summary = summarizer(extractive_summary, max_length=120, min_length=40, do_sample=False)
print("\nFinal Abstractive Summary:\n", final_summary[0]['summary_text'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


This product so far has not disappointed. children love to use it and like the ability to monitor control what content they see with ease. great for beginner or experienced person. Bought as a gift and she loves it Inexpensive tablet for him to use and learn on, step up from the NABI. He was thrilled with it, learn how to Skype on it already... bought this for grand daughter when she comes over to visit. set it up with her as the user, entered her age and name and now makes sure that she only accesses sites and content that are appropriate to her age. Simple to do and she loves the capabilities. also bought and installed a 64gig SD card which gives this little tablet plenty of storage. For the price think this tablet is best one out there. You can spend hundreds of dollars more for additional speed and capacity but when it comes to the basics this tablets does everything that most people will ever need at a fraction of the cost. This fire 8 inch tablet is the perfect size. purchased it

NameError: name 'PeftModel' is not defined

In [113]:
!pip install evaluate --quiet
!pip install rouge_score --quiet
!pip install bert_score --quiet
from evaluate import load

predictions = ["Screen is nice enough and price was right. Works really well with account to access multiple features with extreme ease. 've enjoyed the ease of use and the price paid was great. The price was great and the product is nice."]
references = ["The tablet works well, is easy to use, and offers good value for its price."]

# ROUGE
rouge = load("rouge")
rouge_results = rouge.compute(predictions=predictions, references=references)
print("ROUGE:", rouge_results)

# BERTScore
bertscore = load("bertscore")
bert_results = bertscore.compute(predictions=predictions, references=references, lang="en")
print("BERTScore:", bert_results)

ROUGE: {'rouge1': np.float64(0.2857142857142857), 'rouge2': np.float64(0.03703703703703704), 'rougeL': np.float64(0.21428571428571425), 'rougeLsum': np.float64(0.21428571428571425)}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: {'precision': [0.8687559366226196], 'recall': [0.8964695334434509], 'f1': [0.8823952078819275], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.57.1)'}


In [114]:
# from transformers import BartTokenizer, BartForConditionalGeneration

# # Load tokenizer & model
# tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
# model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Model and tokenizer loaded successfully!")


In [115]:
# import pandas as pd
# from datasets import Dataset

# data = {
#     "reviews.text": [
#         "The phone is great but the battery drains quickly.",
#         "Amazing camera quality and display. Totally worth it!",
#         "Software crashes often. Not recommended."
#     ],
#     "summary": [
#         "Good phone, poor battery.",
#         "Great camera and display.",
#         "Buggy software, not recommended."
#     ]
# }

# df_set = pd.DataFrame(data)
# dataset = Dataset.from_pandas(df_set)


In [116]:
# print(dataset.shape)

In [117]:
# def preprocess_function(examples):
#     model_inputs = tokenizer(
#         examples["reviews.text"],
#         max_length=512,
#         truncation=True
#     )
#     labels = tokenizer(
#         examples["summary"],
#         max_length=64,
#         truncation=True
#     )
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [118]:
# from transformers import Trainer, TrainingArguments

# training_args = TrainingArguments(
#     output_dir="./review_summarizer",
#     # evaluation_strategy="no",      # change to "epoch" for larger data
#     learning_rate=2e-5,
#     per_device_train_batch_size=2,
#     num_train_epochs=1,
#     weight_decay=0.01,
#     save_total_limit=1,
#     logging_dir="./logs",
# )


In [119]:
# from transformers import DataCollatorForSeq2Seq
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset,
#     data_collator=data_collator,  # ✅ this fixes the ValueError
# )

# trainer.train()


In [120]:
# import torch
# torch.cuda.is_available()
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

In [121]:
# model.save_pretrained("./fine_tuned_bart_review_summarizer")
# tokenizer.save_pretrained("./fine_tuned_bart_review_summarizer")


In [122]:
# model = BartForConditionalGeneration.from_pretrained("./fine_tuned_bart_review_summarizer").to(device)
# tokenizer = BartTokenizer.from_pretrained("./fine_tuned_bart_review_summarizer")


In [123]:
# all_reviews_text =  " ".join(df['reviews.text'].dropna().astype(str))

# inputs = tokenizer([all_reviews_text], max_length=512, return_tensors="pt", truncation=True)
# summary_ids = model.generate(inputs["input_ids"].to(device), max_length=60, min_length=10, num_beams=4, early_stopping=True, length_penalty=1.0)
# print("Generated Summary:", tokenizer.decode(summary_ids[0], skip_special_tokens=True))


In [124]:
# !zip -r fine_tuned_bart_review_summarizer.zip fine_tuned_bart_review_summarizer


In [125]:
!pip install transformers peft accelerate datasets


In [132]:
from transformers import BartTokenizer, BartForConditionalGeneration
from peft import LoraConfig, get_peft_model

model_name = "facebook/bart-large-cnn"

tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Configure LoRA
lora_config = LoraConfig(
    r=8,                # rank of LoRA matrices
    lora_alpha=16,      # scaling factor
    target_modules=["q_proj", "v_proj"],  # typical for transformer layers
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"  # because BART is encoder-decoder
)

# Wrap the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 407,470,080 || trainable%: 0.2895


In [134]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")
train_text = dataset["train"][0]["article"]
train_summary = dataset["train"][0]["highlights"]


In [135]:
print(train_summary)

Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .
Young actor says he has no plans to fritter his cash away .
Radcliffe's earnings from first five Potter films have been held in trust fund .


In [136]:
model.save_pretrained("lora-bart-cnn-adapter")


In [140]:
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import numpy as np
import nltk
from peft import PeftModel

nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# -------------------------------
# Step 1: Extractive Summarization
# -------------------------------
prompt_text = (
    "Add product name and Summarize the following review in 3 sentences. "
    "Write in an objective tone and do not use first-person words like 'I' or 'we'. "
    "Focus on the main points and overall sentiment."
)

text = " ".join(df['reviews.text'].dropna().astype(str))
print(text)

# Split into sentences
sentences = sent_tokenize(text)



# Get sentence embeddings
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')
sentence_embeddings = embedder.encode(sentences, convert_to_tensor=True)

# Compute similarity with the "document centroid"
# document_embedding = np.mean(sentence_embeddings.cpu().numpy(), axis=0)
document_embedding = sentence_embeddings.mean(dim=0, keepdim=True)

cosine_scores = util.cos_sim(sentence_embeddings, document_embedding)

# Pick top-N most representative sentences
top_n = 20
top_sentence_indices = np.argsort(-cosine_scores.squeeze().cpu().numpy())[:top_n]
selected_sentences = [sentences[i] for i in sorted(top_sentence_indices)]

extractive_summary = " ".join(selected_sentences)
print("Extractive Summary:\n", extractive_summary)

# -------------------------------
# Step 2: Abstractive Summarization
# -------------------------------
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
base_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
model = PeftModel.from_pretrained(base_model, "lora-bart-cnn-adapter")
summarizer = pipeline("summarization", model=model,tokenizer=tokenizer)

final_summary = summarizer(extractive_summary, max_length=120, min_length=40, do_sample=False)
print("\nFinal Abstractive Summary:\n", final_summary[0]['summary_text'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


This product so far has not disappointed. children love to use it and like the ability to monitor control what content they see with ease. great for beginner or experienced person. Bought as a gift and she loves it Inexpensive tablet for him to use and learn on, step up from the NABI. He was thrilled with it, learn how to Skype on it already... bought this for grand daughter when she comes over to visit. set it up with her as the user, entered her age and name and now makes sure that she only accesses sites and content that are appropriate to her age. Simple to do and she loves the capabilities. also bought and installed a 64gig SD card which gives this little tablet plenty of storage. For the price think this tablet is best one out there. You can spend hundreds of dollars more for additional speed and capacity but when it comes to the basics this tablets does everything that most people will ever need at a fraction of the cost. This fire 8 inch tablet is the perfect size. purchased it

Device set to use cuda:0



Final Abstractive Summary:
 This is a great, easy to use tablet for the price. Works really well with account to access multiple features with extreme ease. The price was good and it has plenty of acc. great tablet, simple to use, just love it.


In [141]:
!zip -r lora-bart-cnn-adapter.zip lora-bart-cnn-adapter

  adding: lora-bart-cnn-adapter/ (stored 0%)
  adding: lora-bart-cnn-adapter/README.md (deflated 66%)
  adding: lora-bart-cnn-adapter/adapter_model.safetensors (deflated 55%)
  adding: lora-bart-cnn-adapter/adapter_config.json (deflated 56%)
